### 1-1. 수리모형 기반 풀이(MTZ)

In [3]:
from ortools.linear_solver import pywraplp
solver=pywraplp.Solver.CreateSolver('SCIP')
def create_model():
    data={}
    data['distance']=[[0,125,225,155,215],
                      [125,0,85,115,135],
                      [225,85,0,165,190],
                      [155,112,165,0,195],
                      [215,135,190,195,0]]
    data['city']=['Basin','Wald','Bon','Mena','Kiln']
    data['vehicle']=1
    data['depot']=0
    return data
    
def print_solution(data,Data,x,status):
    if status==pywraplp.Solver.OPTIMAL:
        print(f'Total distance = {solver.Objective().Value()}m')
        for i in range(Data):
            for j in range(Data):
                if i!=j and x[i,j].solution_value() !=0:
                    print(data['city'][i],'->',data['city'][j])

def main():
    data=create_model()
    x={}
    u={}
    Data=len(data['distance'])
    for i in range(Data):
        u[i]=solver.IntVar(0,solver.infinity(),'u[%i]'%i)
        for j in range(Data):
            if i!=j:
                x[i,j]= solver.BoolVar(f'x[{i},{j}]')
    
    for i in range(Data):
        solver.Add(sum(x[i,j] for j in range(Data) if i!=j)==1)
    for j in range(Data):
        solver.Add(sum(x[i,j] for i in range(Data) if i!=j)==1)
    for i in range(1,Data):
        for j in range(1,Data):
            if i!=j:
                solver.Add(u[i]-u[j]+Data*x[i,j]<=(Data-1))
    solver.Add(u[0]==0)
    obj=[]
    for i in range(Data):
        for j in range(Data):
            if i!=j:
                obj.append(data['distance'][i][j]*x[i,j])
    solver.Minimize(sum(obj))
    status=solver.Solve()
    return print_solution(data,Data,x,status)

if __name__=='__main__':
    main()

Total distance = 750.0m
Basin -> Mena
Wald -> Basin
Bon -> Wald
Mena -> Kiln
Kiln -> Bon


### 1-2 Ortools code 활용 풀이 

In [4]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
def create_model():
    data={}
    M=1000000
    data['distance']=[[0,125,225,155,215],
                      [125,0,85,115,135],
                      [225,85,0,165,190],
                      [155,112,165,0,195],
                      [215,135,190,195,0]]
    data['vehicle']=1
    data['depot']=0
    return data

def print_solution(manager,routing,solution):
    print(f'Total distance : {solution.ObjectiveValue()}m')
    index=routing.Start(0)
    plan_output =f'Routing of Vehicle 0 :\n'
    route_distance=0
    while not routing.IsEnd(index):
        plan_output+=f'{manager.IndexToNode(index)}->'
        previous_index=index
        index=solution.Value(routing.NextVar(index))
        route_distance+=routing.GetArcCostForVehicle(
                previous_index,index,0
            )
    plan_output+=f'{manager.IndexToNode(index)}\n'
    plan_output+=f'Distance of Route: {route_distance}m'
    print(plan_output)

def main():
    data=create_model()
    manager=pywrapcp.RoutingIndexManager(
        len(data['distance']),data['vehicle'],data['depot']
    )
    routing=pywrapcp.RoutingModel(manager)

    def distance_callback(from_index,to_index):
        from_node=manager.IndexToNode(from_index)
        to_node=manager.IndexToNode(to_index)
        return data['distance'][from_node][to_node]
    transit_callback_index=routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameter=pywrapcp.DefaultRoutingSearchParameters()
    search_parameter.first_solution_strategy= routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC


    solution=routing.SolveWithParameters(search_parameter)
    if solution:
        print_solution(manager,routing,solution)


if __name__=='__main__':
    main()

Total distance : 750m
Routing of Vehicle 0 :
0->3->4->2->1->0
Distance of Route: 750m


### 1-3. Nearest Neighbor 경로 계산   
출발: Basin   
->Wald 선택(거리:125)   
->Bon 선택(거리: 85)   
->Mena 선택(거리: 165)   
->Kiln 선택(거리: 195)   
->Basin 복귀(거리: 215)   

In [5]:
print(f'Total distance: {125+85+165+195+215}m')

Total distance: 785m


In [6]:
M = 100000

data = [
          [M, 125, 225, 155, 215],
          [125, M, 85, 115, 135],
          [225, 85, M, 165, 190],
          [155, 115, 165, M, 195],
          [215, 135, 190, 195, M]
        ]

nCity = len(data)

route = [0]
visited = [0]*nCity 

from_node = 0
visited[from_node] = 1
tlength = 0

for _ in range(nCity- 1) :
    distances = data[from_node]
    minDist = M
    minidx = 0
    for i in range(len(distances)) :
        #방문하지 않은 도시 중 minDist를 업데이트해가며 가장 짧은 경로가 저장되게 한다.
        if visited[i] == 0 and distances[i] < minDist :
            minidx = i
            minDist = distances[i]
            
    route.append(minidx)
    tlength = minDist + tlength
    from_node = minidx
    visited[minidx] = 1
    
print('route = ', route)
print('total length = ', tlength + data[from_node][0])

route =  [0, 1, 2, 3, 4]
total length =  785


2. OPEN TOUR 문제

In [27]:
from ortools.linear_solver import pywraplp
M=1000000
Dist=[
    [M,20,30,25,12,33,44,57,0],
    [22,M,19,20,20,29,43,45,0],
    [28,19,M,17,35,48,55,60,0],
    [25,20,19,M,28,35,40,55,0],
    [12,18,34,25,M,21,30,40,0],
    [35,25,45,30,20,M,25,39,0],
    [47,39,50,35,28,20,M,28,0],
    [60,38,54,50,33,40,25,M,0],
    [0,0,0,0,0,0,0,0,M]
    ]
nCity=len(Dist)
solver=pywraplp.Solver.CreateSolver('SAT')

x={}
for i in range(nCity):
    for j in range(nCity):
        if i!=j:
            x[i,j]=solver.BoolVar('x'+str(i)+str(j))
u={}
for i in range(nCity-1):
    u[i]=solver.IntVar(1,nCity-1,'u[%i]'%i)
for j in range(nCity):
    if j==8:
        solver.Add(sum(x[i,j] for i in range(nCity) if i!=j)<=1)
    else:
        solver.Add(sum(x[i,j] for i in range(nCity) if i!=j)==1)
for i in range(nCity):
    if i==8:
        solver.Add(sum(x[i,j] for j in range(nCity) if i!=j)<=1)
    else:
        solver.Add(sum(x[i,j] for j in range(nCity) if i!=j)==1)
#방문순서제약
for i in range(nCity-1):
    for j in range(nCity-1):
        if i!=j:
            solver.Add(u[i]-u[j]+(nCity-1)*x[i,j]<=nCity-2)
obj=[]
for i in range(nCity):
    for j in range(nCity):
        if i!=j:
            obj.append(Dist[i][j]*x[i,j])
solver.Minimize(sum(obj))
status=solver.Solve()
if status==pywraplp.Solver.OPTIMAL or status==pywraplp.Solver.FEASIBLE:
    print(f'Total cost = {solver.Objective().Value():.1f}\n')
    for i in range(nCity-1):
        print(f'{i}방문순서: {u[i].solution_value():.0f}')
else:
    print('No solution found')


Total cost = 133.0

0방문순서: 5
1방문순서: 6
2방문순서: 7
3방문순서: 8
4방문순서: 4
5방문순서: 3
6방문순서: 2
7방문순서: 1
